In [1]:
import ase.io as aio

import os
import numpy as np
import qml
import ase
import sys
sys.path.insert(0, '/home/sahre/git_repositories/APDFT/prototyping/atomic_energies/hitp/')
sys.path.insert(0, '/home/sahre/git_repositories/APDFT/prototyping/atomic_energies/')
from prepare_calculations import *
import explore_qml_data as eqd
import utils_qm as uqm
import glob

In /home/sahre/opt/miniconda3/envs/qm-tools/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/sahre/opt/miniconda3/envs/qm-tools/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/sahre/opt/miniconda3/envs/qm-tools/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/sahre/opt/miniconda3/envs/qm-tools/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor rele

In [43]:
def wrapper_ase2(atoms, compound_path, lambda_values, pp_dir, pp_type, template_inp, template_inp_small_lambda):
    """
    generates all necessary files for a cpmd calculation using an atoms object from ase as input
    """
    # calculation parameters (independent of lambda value)
    atom_symbols = atoms.get_chemical_symbols()
    nuc_charges = atoms.get_atomic_numbers()
    num_ve = eqd.get_num_val_elec(nuc_charges) # get number of ve
    boxsize = get_boxsize(num_ve) # get boxsize
    num_gpts_lower, num_gpts_higher = get_gpts(num_ve/2) # get gridpoints
    num_gpts = num_gpts_higher
    # shift molecule to center of box
    coords_final = eqd.shift2center(atoms.get_positions(), np.array([boxsize, boxsize, boxsize])/2)

    # get correct lambda value
    for lam_val in lambda_values:
        #print(lam_val)
        elec_pp = np.round(lam_val*num_ve) # electrons coming from pseudopotentials
        charge = elec_pp-num_ve # electrons added to conserve total number of electrons

        lambda_path = os.path.join(compound_path, f'lam_{lam_val}/')
        os.makedirs(lambda_path, exist_ok=True)

        # generate input file
        input_path = os.path.join(lambda_path, 'run.inp')
        if lam_val > 0.501:
            write_input(atom_symbols, charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp, debug = False)
        else:
            write_input(atom_symbols, charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp_small_lambda, debug = False)

        # generate pp-files
        write_pp_files_compound(atom_symbols, lam_val, lambda_path, pp_dir, pp_type)
        
        print(lambda_path)
        
def wrapper_ase3(atoms, compound_path, lambda_values, pp_dir, pp_type, template_inp, template_inp_small_lambda):
    """
    generates all necessary files for a cpmd calculation using an atoms object from ase as input
    """
    # calculation parameters (independent of lambda value)
    atom_symbols = ['H1', 'H2']#atoms.get_chemical_symbols()
    nuc_charges = atoms.get_atomic_numbers()
    num_ve = eqd.get_num_val_elec(nuc_charges) # get number of ve
    boxsize = get_boxsize(num_ve) # get boxsize
    num_gpts_lower, num_gpts_higher = get_gpts(num_ve/2) # get gridpoints
    num_gpts = num_gpts_higher
    # shift molecule to center of box
    coords_final = eqd.shift2center(atoms.get_positions(), np.array([boxsize, boxsize, boxsize])/2)

    # get correct lambda value
    for lam_val1, lam_val2 in lambda_values:
        #print(lam_val)
        elec_pp = int(np.round(lam_val1 + lam_val2)) # electrons coming from pseudopotentials
        charge = elec_pp-num_ve # electrons added to conserve total number of electrons
        lambda_path = os.path.join(compound_path, f'H2_lam_{lam_val2}/')
        os.makedirs(lambda_path, exist_ok=True)

        # generate input file
        input_path = os.path.join(lambda_path, 'run.inp')
        if lam_val1 > 0.501:
            write_input(atom_symbols, charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp, debug = False)
        else:
            write_input(atom_symbols, charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp_small_lambda, debug = False)

        # generate pp-files
        #write_pp_files_compound(atom_symbols, lam_val, lambda_path, pp_dir, pp_type)
        write_pp_files_compound_partial(atom_symbols, [lam_val1, lam_val2], lambda_path, pp_dir, pp_type)
        print(lambda_path)

In [48]:
# define parameters
pp_dir = '/data/sahre/PP_LIBRARY'
pp_type = 'SG_LDA'
template_inp = '/data/sahre/projects/atomic-energies/cpmd_params_template_lda_cutoff.inp'
template_inp_small_lambda = '/data/sahre/projects/atomic-energies/cpmd_params_template_lda_small_lambda_cutoff.inp'
compound_path = '/data/sahre/projects/atomic-energies/derivatives/H2/H2_stretched'
dlam = 0.00001
#lambda_values = np.array([0.11, 0.31, 0.41, 0.51, 0.71, 0.91]) + dlam

lambda_values = np.sort(np.concatenate((np.arange(0.0, 1.05, 0.05), np.arange(0.0, 1.05, 0.05)+dlam)))

atoms = aio.read('/data/sahre/projects/atomic-energies/derivatives/H2/H2_stretched.xyz')

wrapper_ase2(atoms, compound_path, lambda_values, pp_dir, pp_type, template_inp, template_inp_small_lambda)

/data/sahre/projects/atomic-energies/derivatives/H2/H2_stretched/lam_0.0/
/data/sahre/projects/atomic-energies/derivatives/H2/H2_stretched/lam_1e-05/
/data/sahre/projects/atomic-energies/derivatives/H2/H2_stretched/lam_0.05/
/data/sahre/projects/atomic-energies/derivatives/H2/H2_stretched/lam_0.050010000000000006/
/data/sahre/projects/atomic-energies/derivatives/H2/H2_stretched/lam_0.1/
/data/sahre/projects/atomic-energies/derivatives/H2/H2_stretched/lam_0.10001/
/data/sahre/projects/atomic-energies/derivatives/H2/H2_stretched/lam_0.15000000000000002/
/data/sahre/projects/atomic-energies/derivatives/H2/H2_stretched/lam_0.15001000000000003/
/data/sahre/projects/atomic-energies/derivatives/H2/H2_stretched/lam_0.2/
/data/sahre/projects/atomic-energies/derivatives/H2/H2_stretched/lam_0.20001000000000002/
/data/sahre/projects/atomic-energies/derivatives/H2/H2_stretched/lam_0.25/
/data/sahre/projects/atomic-energies/derivatives/H2/H2_stretched/lam_0.25001/
/data/sahre/projects/atomic-energie

In [52]:
# define parameters
pp_dir = '/data/sahre/PP_LIBRARY'
pp_type = 'SG_LDA'
template_inp = '/data/sahre/projects/atomic-energies/cpmd_params_template_lda_cutoff.inp'
template_inp_small_lambda = '/data/sahre/projects/atomic-energies/cpmd_params_template_lda_small_lambda_cutoff.inp'
compound_path = '/data/sahre/projects/atomic-energies/derivatives/H2/partial_derivatives_stretched/'
dlam = 0.00001

lam = np.arange(0.0, 1.05, 0.05)

lambda_values = np.array([lam, lam+dlam]).T

atoms = aio.read('/data/sahre/projects/atomic-energies/derivatives/H2/H2_stretched.xyz')

wrapper_ase3(atoms, compound_path, lambda_values, pp_dir, pp_type, template_inp, template_inp_small_lambda)

/data/sahre/projects/atomic-energies/derivatives/H2/partial_derivatives_stretched/H2_lam_1e-05/
/data/sahre/projects/atomic-energies/derivatives/H2/partial_derivatives_stretched/H2_lam_0.050010000000000006/
/data/sahre/projects/atomic-energies/derivatives/H2/partial_derivatives_stretched/H2_lam_0.10001/
/data/sahre/projects/atomic-energies/derivatives/H2/partial_derivatives_stretched/H2_lam_0.15001000000000003/
/data/sahre/projects/atomic-energies/derivatives/H2/partial_derivatives_stretched/H2_lam_0.20001000000000002/
/data/sahre/projects/atomic-energies/derivatives/H2/partial_derivatives_stretched/H2_lam_0.25001/
/data/sahre/projects/atomic-energies/derivatives/H2/partial_derivatives_stretched/H2_lam_0.30001000000000005/
/data/sahre/projects/atomic-energies/derivatives/H2/partial_derivatives_stretched/H2_lam_0.35001000000000004/
/data/sahre/projects/atomic-energies/derivatives/H2/partial_derivatives_stretched/H2_lam_0.40001000000000003/
/data/sahre/projects/atomic-energies/derivative

In [49]:
np.arange(0.0, 1.05, 0.05)

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])